<a href="https://colab.research.google.com/github/ramincre/Dual-Path-Transformer-Network-PyTorch/blob/main/BSS_DPTNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
import torchvision.models as models
from torch.profiler import profile, record_function, ProfilerActivity

In [3]:
!cp "/content/drive/MyDrive/Dataset/MiniLibriMix.rar" "/content"

In [ ]:
!unrar x /content/MiniLibriMix.rar

In [ ]:
import os
import sys

In [ ]:
import torch
torch.cuda.is_available()

In [8]:
!git clone https://github.com/ramincre/Dual-Path-Transformer-Network-PyTorch.git

Cloning into 'Dual-Path-Transformer-Network-PyTorch'...
remote: Enumerating objects: 301, done.
remote: Counting objects: 100% (139/139), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 301 (delta 68), reused 129 (delta 63), pack-reused 162
Receiving objects: 100% (301/301), 2.62 MiB | 5.20 MiB/s, done.
Resolving deltas: 100% (137/137), done.


In [ ]:
!mkdir /content/output

In [ ]:
!python /content/Dual-Path-Transformer-Network-PyTorch/preprocess.py --in-dir /content/MiniLibriMix-Clean --out-dir /content/output --sample-rate 8000

In [ ]:
!python /content/Dual-Path-Transformer-Network-PyTorch/train.py \
--train_dir '/content/output/tr'\
--valid_dir '/content/output/cv'\
--epochs 20 \
--batch_size 2 \
--num_workers 0  \
--N 48\
--momentum 0.01\
--l2 0.001

In [ ]:
#import os
#os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

In [ ]:
!pip install mir_eval

In [ ]:
!python /content/Dual-Path-Transformer-Network-PyTorch/evaluate.py \
--data_dir '/content/output/tt' \
--model_path '/content/exp/temp/temp_best.pth.tar'\
--batch_size 2\
--epochs 20 \
--batch_size 2 \
--num_workers 0  \
--N 48\
--momentum 0.01\


In [ ]:
!python /content/Dual-Path-Transformer-Network-PyTorch/separate.py\
 --mix_json '/content/output/tt/mix.json' \
 --model_path '/content/exp/temp/temp_best.pth.tar'\
 --N 48 \
--batch_size 2

In [6]:
%%bash
pip install torchmetrics
pip install pystoi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 8.3 MB/s eta 0:00:00


In [7]:
import io
import os
import math
import tarfile
import multiprocessing
import sys
import scipy
import librosa
import random
import requests
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import time
import numpy as np
import IPython.display as ipd
import torch
import torchaudio
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import time

from timeit import default_timer as timer
from IPython.display import Audio, display
from scipy import signal
from scipy.spatial import distance
from scipy.io import wavfile
from torch.nn.functional import normalize
from torch.profiler import profile, record_function, ProfilerActivity
from torchmetrics import ScaleInvariantSignalNoiseRatio
from torchmetrics.audio import SignalDistortionRatio
from torchmetrics import ScaleInvariantSignalDistortionRatio
from torchmetrics.audio.stoi import ShortTimeObjectiveIntelligibility

In [9]:
!git clone https://github.com/tky823/DNN-based_source_separation.git

Cloning into 'DNN-based_source_separation'...
remote: Enumerating objects: 22830, done.
remote: Counting objects: 100% (893/893), done.
remote: Compressing objects: 100% (271/271), done.
remote: Total 22830 (delta 579), reused 854 (delta 574), pack-reused 21937
Receiving objects: 100% (22830/22830), 300.24 MiB | 21.60 MiB/s, done.
Resolving deltas: 100% (15673/15673), done.
Updating files: 100% (711/711), done.


In [10]:
sys.path.append("/content/DNN-based_source_separation/src/")

In [11]:
from models.lstm_tasnet import LSTMTasNet
from models.conv_tasnet import ConvTasNet
from models.dprnn_tasnet import DPRNNTasNet
from models.dptnet import DPTNet

In [12]:
waveform_orginal_0, sample_rate = torchaudio.load("/content/MiniLibriMix-Clean/tt/s1/100-121669-0026_718-129597-0003.wav")
waveform_orginal_1, sample_rate = torchaudio.load("/content/MiniLibriMix-Clean/tt/s2/100-121669-0026_718-129597-0003.wav")

SAMPLE_RATE_WSJ0 = 8000

resampler = torchaudio.transforms.Resample(sample_rate, SAMPLE_RATE_WSJ0)
waveform_orginal_0 = resampler(waveform_orginal_0)
waveform_orginal_1 = resampler(waveform_orginal_1)


T_min = min(waveform_orginal_0.size(-1), waveform_orginal_1.size(-1))
waveform_orginal_0, waveform_orginal_1 = waveform_orginal_0[:, :T_min], waveform_orginal_1[:, :T_min]

display(ipd.Audio(waveform_orginal_0, rate=SAMPLE_RATE_WSJ0))
display(ipd.Audio(waveform_orginal_1, rate=SAMPLE_RATE_WSJ0))

In [14]:
n_sources = 2
mixture = waveform_orginal_0 + waveform_orginal_1
display(ipd.Audio(mixture, rate=SAMPLE_RATE_WSJ0))

In [18]:
!mkdir "/content/saved/DPTNet"

mkdir: cannot create directory ‘/content/saved/DPTNet’: No such file or directory


In [15]:
model = DPTNet.build_from_pretrained(task="wsj0-mix", sample_rate=8000, n_sources=2)
model.eval()

input = mixture.unsqueeze(dim=0)
with torch.no_grad():
    output = model(input)

output = output.squeeze(dim=0)
estimated = torch.split(output, [1]*n_sources, dim=0)

for idx in range(n_sources):
    display(ipd.Audio(estimated[idx], rate=SAMPLE_RATE_WSJ0))
    path = f"/content/saved/FixedAttractorDANet/speaker{idx+1}.wav"
    torchaudio.save(path, estimated[idx], SAMPLE_RATE_WSJ0)

Downloading...
From: https://drive.google.com/uc?id=1QJnJEK8aed7_ED07jD7buyGb37giEDUx
To: /tmp/85627923-2266-46d5-ae46-e8f5317c16b2.zip
100%|██████████| 20.9M/20.9M [00:00<00:00, 34.1MB/s]


In [16]:
def DisplaySpectrogram(sp1_pth,sp2_pth,mix_pth):
  samplingFrequency1, signalData1 = wavfile.read(sp1_pth)
  samplingFrequency2, signalData2 = wavfile.read(sp2_pth)
  samplingFrequency3, signalData3 = wavfile.read(mix_pth)

  cmap = plt.get_cmap('viridis')
  cmap.set_under(color='k', alpha=None)

  fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, figsize=(20, 10))
  fig.tight_layout()

  values1, ybins1, xbins1, im1 = ax1.specgram(signalData1, NFFT=512, cmap=cmap, Fs=samplingFrequency1)
  ax1.set(title='Speaker1')

  values2, ybins2, xbins2, im2 = ax2.specgram(signalData2, NFFT=512, cmap=cmap, Fs=samplingFrequency2)
  ax2.set(title='Speaker2')

  values3, ybins3, xbins3, im3 = ax3.specgram(signalData3, NFFT=512, cmap=cmap, Fs=samplingFrequency3)
  ax3.set(title='Mix')

  fig.colorbar(im1, ax=ax1)
  fig.colorbar(im2, ax=ax2)
  fig.colorbar(im3, ax=ax3)

  plt.show()

In [17]:
DisplaySpectrogram(estimated[0],estimated[1],mixture)

TypeError: expected str, bytes or os.PathLike object, not Tensor